In [1]:
import sys; sys.path.append("..")
from policies import max_min_fairness, max_min_fairness_strategy_proof
from strategy_proofness_tests import test_strategy_proofness
import copy
import random
import time

import numpy as np
np.set_printoptions(precision=3, suppress=True)

In [2]:
def print_allocation_analysis(honest_throughputs, dishonest_throughputs,
                              strategy_proof_allocations, non_strategy_proof_allocations):
    print("=" * 100)
    print("Honest throughputs:", np.array(honest_throughputs))
    print("Dishonest throughputs:", np.array(dishonest_throughputs))
    print("-" * 100)

    for allocations, allocation_str in zip(
        [strategy_proof_allocations, non_strategy_proof_allocations],
        ["Strategy-proof", "Not strategy-proof"]):
        for allocation, throughput_str in zip(allocations,
                                              ["Honest throughputs", "Dishonest throughputs"]):

            print("%s (%s)\n" % (allocation_str, throughput_str))
            effective_throughputs = np.sum(np.multiply(np.array(honest_throughputs), allocation),
                                           axis=1)
            print("Allocation:", allocation)
            print("Effective throughputs:", effective_throughputs)
            print("-" * 100)
    print()
    print()

In [3]:
def random_dishonest():
    worker_types = ['v100', 'p100', 'k80']
    cluster_spec = {
            'v100': 1, 'p100': 1, 'k80': 1}
     
    for i in range(100):
        # throughputs specified in the above order.
        honest_throughputs = []
        for j in range(10):
            honest_throughputs.append(sorted([random.random(), random.random(), random.random()],
                                             reverse=True))
        dishonest_throughputs = copy.copy(honest_throughputs)
        dishonest_throughputs[0] = [random.random(), random.random(), random.random()]
        
        strategy_proof_allocations = []
        non_strategy_proof_allocations = []
        for (throughputs, experiment_string) in zip(
            [honest_throughputs, dishonest_throughputs],
            ["Allocation computed using true throughputs",
             "Allocation computed using dishonest throughputs"]):
            
            throughputs = np.array(throughputs)
            unflattened_throughputs = {
                i: {worker_types[j]: throughputs[i][j] for j in range(len(worker_types))}
                for i in range(len(throughputs))
            }
            unflattened_strategy_proof_allocation, discount_factors, \
                unflattened_non_strategy_proof_allocation, runtime = \
                    test_strategy_proofness(unflattened_throughputs, cluster_spec, verbose=False)
            strategy_proof_allocation = np.zeros((len(throughputs), len(cluster_spec)))
            non_strategy_proof_allocation = np.zeros((len(throughputs), len(cluster_spec)))
            for i in range(len(throughputs)):
                for j in range(len(worker_types)):
                    strategy_proof_allocation[i][j] = \
                        unflattened_strategy_proof_allocation[i][worker_types[j]]
                    non_strategy_proof_allocation[i][j] = \
                        unflattened_non_strategy_proof_allocation[i][worker_types[j]]
            
            strategy_proof_allocations.append(strategy_proof_allocation)
            non_strategy_proof_allocations.append(non_strategy_proof_allocation)
            
        effective_throughputs_honest_and_non_strategy_proof_allocation = \
            np.sum(np.multiply(honest_throughputs,
                               non_strategy_proof_allocations[0]), axis=1)
        effective_throughputs_dishonest_and_non_strategy_proof_allocation = \
            np.sum(np.multiply(honest_throughputs,
                               non_strategy_proof_allocations[1]), axis=1)
        effective_throughputs_honest_and_strategy_proof_allocation = \
            np.sum(np.multiply(honest_throughputs,
                               strategy_proof_allocations[0]), axis=1)
        effective_throughputs_dishonest_and_strategy_proof_allocation = \
            np.sum(np.multiply(honest_throughputs,
                               strategy_proof_allocations[1]), axis=1)
        
        if (effective_throughputs_honest_and_strategy_proof_allocation[0] /
               effective_throughputs_dishonest_and_strategy_proof_allocation[0] < 0.99):
            print("WARNING: Strategy-proof allocation not actually strategy-proof!")
            print_allocation_analysis(honest_throughputs, dishonest_throughputs,
                                      strategy_proof_allocations, non_strategy_proof_allocations)
            continue
        
        if (effective_throughputs_honest_and_non_strategy_proof_allocation[0] >=
            effective_throughputs_dishonest_and_non_strategy_proof_allocation[0]):
            continue

        print("SUCCESS: Strategy-proof allocation strategy-proof!")
        print_allocation_analysis(honest_throughputs, dishonest_throughputs,
                                  strategy_proof_allocations, non_strategy_proof_allocations)

In [ ]:
random.seed(42)
random_dishonest()

SUCCESS: Strategy-proof allocation strategy-proof!
Honest throughputs: [[0.669 0.238 0.071]
 [0.936 0.214 0.132]
 [0.785 0.571 0.473]
 [0.807 0.19  0.097]
 [0.467 0.431 0.424]
 [0.984 0.729 0.673]
 [0.403 0.339 0.098]
 [0.862 0.249 0.19 ]
 [0.449 0.422 0.279]
 [0.923 0.443 0.25 ]]
Dishonest throughputs: [[0.861 0.55  0.051]
 [0.936 0.214 0.132]
 [0.785 0.571 0.473]
 [0.807 0.19  0.097]
 [0.467 0.431 0.424]
 [0.984 0.729 0.673]
 [0.403 0.339 0.098]
 [0.862 0.249 0.19 ]
 [0.449 0.422 0.279]
 [0.923 0.443 0.25 ]]
----------------------------------------------------------------------------------------------------
Strategy-proof (Honest throughputs)

Allocation: [[0.082 0.    0.   ]
 [0.082 0.    0.   ]
 [0.    0.109 0.048]
 [0.082 0.    0.   ]
 [0.    0.    0.179]
 [0.    0.    0.179]
 [0.    0.149 0.   ]
 [0.082 0.    0.   ]
 [0.    0.149 0.   ]
 [0.082 0.    0.   ]]
Effective throughputs: [0.055 0.077 0.085 0.066 0.076 0.121 0.05  0.071 0.063 0.076]
--------------------------------------